<a href="https://colab.research.google.com/github/on-vladislav/Video_generation/blob/main/Video_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Библиотеки

In [ ]:
!pip install requests
!pip install moviepy

In [ ]:
# Импорт модулей
import requests
import os
from moviepy.editor import VideoFileClip, CompositeVideoClip, ImageClip

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Код

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('PixabayAPI')


In [ ]:
# Настройки
#PIXABAY_API_KEY = my_secret_key  # Замените на ваш API ключ

search_query = 'Data Scientist'  # Ваш поисковый запрос
per_page = 3  # Количество видео для загрузки

logo_path = '/content/logo.png'  # Путь к вашему логотипу

# Функции
def download_pixabay_videos(query, per_page=3):
    url = 'https://pixabay.com/api/videos/'

    params = {
        'key': my_secret_key,
        'q': query,
        'per_page': per_page,
        'safesearch': 'true'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        videos = response.json()['hits']
        video_paths = []
        for idx, video in enumerate(videos):
            # Выбираем видео с наилучшим качеством
            video_files = video['videos']
            if 'large' in video_files:
                video_url = video_files['large']['url']
            elif 'medium' in video_files:
                video_url = video_files['medium']['url']
            else:
                video_url = video_files['small']['url']

            video_response = requests.get(video_url)
            video_filename = f'video_{idx}.mp4'
            with open(video_filename, 'wb') as f:
                f.write(video_response.content)
            video_paths.append(video_filename)
        return video_paths
    else:
        print('Ошибка при запросе к Pixabay API:', response.status_code)
        return []

def overlay_logo_on_video(video_path, logo_path, output_path):
    # Загрузка видео
    video_clip = VideoFileClip(video_path)

    # Проверка наличия аудио (для избежания предупреждений)
    if video_clip.audio is None:
        video_clip = video_clip.set_audio(None)

    # Загрузка логотипа
    logo = (ImageClip(logo_path)
            .set_duration(video_clip.duration)
            .resize(height=100)  # Измените размер логотипа при необходимости
            .margin(right=8, top=8, opacity=0)  # Отступы
            .set_pos(("right", "top")))  # Позиция логотипа

    # Наложение логотипа на видео
    final_video = CompositeVideoClip([video_clip, logo])

    # Сохранение итогового видео
    final_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # Освобождение ресурсов
    video_clip.close()
    final_video.close()

# Основной процесс

video_paths = download_pixabay_videos(search_query, per_page)

if video_paths:
    for video_path in video_paths:
        output_path = f'logo_{video_path}'
        overlay_logo_on_video(video_path, logo_path, output_path)
        print(f'Видео с наложенным логотипом сохранено: {output_path}')
else:
    print('Не удалось скачать видео.')

Moviepy - Building video logo_video_0.mp4.
Moviepy - Writing video logo_video_0.mp4



Moviepy - Done !
Moviepy - video ready logo_video_0.mp4
Видео с наложенным логотипом сохранено: logo_video_0.mp4
Moviepy - Building video logo_video_1.mp4.
MoviePy - Writing audio in logo_video_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video logo_video_1.mp4



t: 100%|█████████▉| 381/382 [00:56<00:00,  6.67it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file video_1.mp4, 6220800 bytes wanted but 0 bytes read,at frame 380/382, at time 15.20/15.25 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|██████████| 382/382 [00:56<00:00,  7.04it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file video_1.mp4, 6220800 bytes wanted but 0 bytes read,at frame 381/382, at time 15.24/15.25 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready logo_video_1.mp4
Видео с наложенным логотипом сохранено: logo_video_1.mp4
Moviepy - Building video logo_video_2.mp4.
Moviepy - Writing video logo_video_2.mp4



Moviepy - Done !
Moviepy - video ready logo_video_2.mp4
Видео с наложенным логотипом сохранено: logo_video_2.mp4


Совмещение

In [ ]:
# Импорт модулей
from moviepy.editor import VideoFileClip, clips_array

# Настройки
video_paths = ['/content/logo_video_0.mp4', '/content/logo_video_1.mp4', '/content/logo_video_2.mp4']  # Замените на ваши файлы
output_path = 'merged_video.mp4'

# Загружаем видеофайлы
video_clips = [VideoFileClip(video) for video in video_paths]

target_size = (640, 360)  # Задайте нужный размер
resized_clips = [clip.resize(target_size) for clip in video_clips]

# Объединяем видео в один кадр (в ряд)
final_clip = clips_array([resized_clips])

# Сохраняем итоговое видео
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Освобождаем ресурсы
for clip in resized_clips:
    clip.close()
final_clip.close()

print(f'Итоговое видео сохранено как {output_path}')

Moviepy - Building video merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video merged_video.mp4



Moviepy - Done !
Moviepy - video ready merged_video.mp4
Итоговое видео сохранено как merged_video.mp4
